In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from src.dataset import ChestXrayDataSet, CLASS_NAMES
from src.model import DenseNet121
from src.utils import compute_AUCs
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Autoreload modules so that changes to src automatically reflect
%load_ext autoreload
%autoreload 2

In [2]:
labels = pd.read_csv("data/labels/cleaned.csv")
X, Y = labels.iloc[:, 0], labels.iloc[:, 1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, shuffle=True)

In [3]:
N_CLASSES = len(CLASS_NAMES)
BATCH_SIZE = 64
DATA_DIR = "data/images"

In [4]:
train_dataset = ChestXrayDataSet(data_dir=DATA_DIR, X = X_train, Y = Y_train,
                          transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
                              transforms.RandomHorizontalFlip()
                              ]))

# note that workers take up some amount of VRAM   
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

test_dataset = ChestXrayDataSet(data_dir=DATA_DIR, X = X_test, Y = Y_test,
                          transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                              ]))
    
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [5]:
CKPT_TRAINED_PATH = "model-trained.pth"

cudnn.benchmark = True # Fixed input size, enables tuning for optimal use

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# initialize and load the model
model = DenseNet121(N_CLASSES).to(device)

if os.path.isfile(CKPT_TRAINED_PATH):
    print("=> loading checkpoint")
    checkpoint = torch.load(CKPT_TRAINED_PATH)
    # Load directly into the module else the model gets screwed up
    # https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/15
    model.load_state_dict(checkpoint, strict=True)
    print("=> loaded checkpoint")
else:
    print("=> no checkpoint found")

=> loading checkpoint
=> loaded checkpoint


In [6]:
# initialize the ground truth and output tensor
gt = torch.FloatTensor()
pred = torch.FloatTensor()
gt = gt.cuda()
pred = pred.cuda()

# switch to evaluate mode
model.eval()

for i, (inp, target) in enumerate(test_loader):
    target = target.cuda()
    gt = torch.cat((gt, target), 0)
    bs, c, h, w = inp.size()
    with torch.no_grad():
        input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda())
        output = model(input_var)
        output_mean = output.view(bs, 1, -1).mean(1)
        pred = torch.cat((pred, output_mean.data), 0)

In [7]:
AUROCs = compute_AUCs(gt, pred, N_CLASSES)
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
  print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

The average AUROC is 0.837
The AUROC of Atelectasis is 0.8445321318826039
The AUROC of Cardiomegaly is 0.9553322827202491
The AUROC of Effusion is 0.8845303803713797
The AUROC of Infiltration is 0.7769573854413471
The AUROC of Mass is 0.8597052845983459
The AUROC of Nodule is 0.8158398278852277
The AUROC of Pneumonia is 0.6880329089733861
The AUROC of Pneumothorax is 0.8771257764586765
The AUROC of Consolidation is 0.7412101813643776
The AUROC of Edema is 0.8966185637985037
The AUROC of Emphysema is 0.8714209142016629
The AUROC of Fibrosis is 0.825740948305866
The AUROC of Pleural_Thickening is 0.743689645140377
The AUROC of Hernia is 0.9400009377184458
